In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import librosa.display
import cv2
import matplotlib.pyplot as plt
import time
import pickle
import glob
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
# import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from resnet import ResNet, Bottleneck

from albumentations.core.transforms_interface import DualTransform, BasicTransform
import albumentations as albu

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
INPUT_ROOT = Path("/home/knikaido/work/Rainforest-Connection/data")
RAW_DATA = INPUT_ROOT / "rfcx-species-audio-detection"
TRAIN_AUDIO_DIR = RAW_DATA / "train"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test"
FEAT_DIR =  '/home/knikaido/work/Rainforest-Connection/Git/feature/18/'

In [41]:
train_df = pd.read_pickle(RAW_DATA / "train_gby_wav_raw.pkl")
train_df

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25,/home/knikaido/work/Rainforest-Connection/data...
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40,/home/knikaido/work/Rainforest-Connection/data...
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25,/home/knikaido/work/Rainforest-Connection/data...
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04,/home/knikaido/work/Rainforest-Connection/data...
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70,/home/knikaido/work/Rainforest-Connection/data...
...,...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75,/home/knikaido/work/Rainforest-Connection/data...
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25,/home/knikaido/work/Rainforest-Connection/data...
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00,/home/knikaido/work/Rainforest-Connection/data...
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20,/home/knikaido/work/Rainforest-Connection/data...


In [42]:
len(y)

2880000

In [43]:
sizes = []
for i in range(len(train_df['name'])):
    y, sr = sf.read(train_df['name'][i])
    sizes.append(np.max(y[int(train_df['t_min'][i]*sr):int(train_df['t_max'][i]*sr)]))
    
#     break

In [44]:
train_df['size'] = sizes
train_df

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name,size
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25,/home/knikaido/work/Rainforest-Connection/data...,0.064270
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40,/home/knikaido/work/Rainforest-Connection/data...,0.106873
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25,/home/knikaido/work/Rainforest-Connection/data...,0.333618
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04,/home/knikaido/work/Rainforest-Connection/data...,0.038879
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70,/home/knikaido/work/Rainforest-Connection/data...,0.151276
...,...,...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75,/home/knikaido/work/Rainforest-Connection/data...,0.098907
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25,/home/knikaido/work/Rainforest-Connection/data...,0.650818
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00,/home/knikaido/work/Rainforest-Connection/data...,0.201111
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20,/home/knikaido/work/Rainforest-Connection/data...,0.059113


In [45]:
train_df.sort_values('size')

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name,size
49,0a350d11c,19,1,11.5253,281.25,13.0773,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.016296
1024,d60157507,20,1,20.4427,1312.50,25.8240,6093.75,/home/knikaido/work/Rainforest-Connection/data...,0.016907
227,2e40b2294,0,1,56.7307,5906.25,58.0213,7500.00,/home/knikaido/work/Rainforest-Connection/data...,0.017120
523,69ea6d76b,19,1,30.9333,281.25,32.4853,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.017853
990,cfdd68da2,19,1,7.7440,281.25,9.2960,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.019012
...,...,...,...,...,...,...,...,...,...
474,5f8eecc9e,23,4,51.0084,6459.96,58.9322,11627.90,/home/knikaido/work/Rainforest-Connection/data...,0.999756
475,5f8eecc9e,9,1,54.9440,1125.00,57.7493,5250.00,/home/knikaido/work/Rainforest-Connection/data...,0.999756
138,1ca8c8af6,12,1,0.6827,562.50,3.8187,3281.25,/home/knikaido/work/Rainforest-Connection/data...,0.999939
477,5f8eecc9e,23,4,7.9412,6459.96,15.8650,11627.90,/home/knikaido/work/Rainforest-Connection/data...,0.999969


In [46]:
train_df[train_df['size'] < 0.02]

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name,size
49,0a350d11c,19,1,11.5253,281.250,13.0773,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.016296
105,163e86660,3,1,58.0640,1031.250,58.8053,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.019257
227,2e40b2294,0,1,56.7307,5906.250,58.0213,7500.00,/home/knikaido/work/Rainforest-Connection/data...,0.017120
523,69ea6d76b,19,1,30.9333,281.250,32.4853,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.017853
666,87b64a134,19,1,34.0480,281.250,35.6000,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.019470
823,ae210f726,3,1,44.1333,1031.250,44.8747,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.019775
853,b41ae319b,6,1,4.5013,562.500,6.6453,4406.25,/home/knikaido/work/Rainforest-Connection/data...,0.019104
990,cfdd68da2,19,1,7.7440,281.250,9.2960,2812.50,/home/knikaido/work/Rainforest-Connection/data...,0.019012
1013,d5163b4ef,10,1,5.5554,947.461,7.9993,10852.70,/home/knikaido/work/Rainforest-Connection/data...,0.019745
1024,d60157507,20,1,20.4427,1312.500,25.8240,6093.75,/home/knikaido/work/Rainforest-Connection/data...,0.016907


In [47]:
train_df = train_df[train_df['size'] > 0.02]
train_df.reset_index(inplace=True, drop=True)
train_df

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name,size
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25,/home/knikaido/work/Rainforest-Connection/data...,0.064270
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40,/home/knikaido/work/Rainforest-Connection/data...,0.106873
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25,/home/knikaido/work/Rainforest-Connection/data...,0.333618
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04,/home/knikaido/work/Rainforest-Connection/data...,0.038879
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70,/home/knikaido/work/Rainforest-Connection/data...,0.151276
...,...,...,...,...,...,...,...,...,...
1201,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75,/home/knikaido/work/Rainforest-Connection/data...,0.098907
1202,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25,/home/knikaido/work/Rainforest-Connection/data...,0.650818
1203,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00,/home/knikaido/work/Rainforest-Connection/data...,0.201111
1204,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20,/home/knikaido/work/Rainforest-Connection/data...,0.059113


In [48]:
train_df.to_pickle(str(RAW_DATA / 'train_gby_wav_raw_small.pkl'))

In [49]:
# tgt_path_list = []
# for i in tqdm(range(len(train))):
#     for j in range(len(wav_pathes)):
#         if train['recording_id'][i] in wav_pathes[j]:
#             tgt_path_list.append(wav_pathes[j])

In [50]:
tat_path_np = np.array(wav_pathes)

NameError: name 'wav_pathes' is not defined

In [7]:
# tat_path_list_count = pd.Series(tat_path_np).value_counts().index.tolist()

In [8]:
tat_path_list_df = pd.Series(tat_path_np)

In [9]:
len(tat_path_list_df)

1132

In [10]:
tat_path_list_df.name = 'path'
tat_path_list_df

0       /home/knikaido/work/Rainforest-Connection/Git/...
1       /home/knikaido/work/Rainforest-Connection/Git/...
2       /home/knikaido/work/Rainforest-Connection/Git/...
3       /home/knikaido/work/Rainforest-Connection/Git/...
4       /home/knikaido/work/Rainforest-Connection/Git/...
                              ...                        
1127    /home/knikaido/work/Rainforest-Connection/Git/...
1128    /home/knikaido/work/Rainforest-Connection/Git/...
1129    /home/knikaido/work/Rainforest-Connection/Git/...
1130    /home/knikaido/work/Rainforest-Connection/Git/...
1131    /home/knikaido/work/Rainforest-Connection/Git/...
Name: path, Length: 1132, dtype: object

In [11]:
train.sort_values('recording_id').head(20)

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70
5,00b404881,8,1,0.0747,3750.000,4.1973,5531.25
6,00d442df7,0,1,19.3653,5906.250,20.1600,8250.00
7,011f25080,18,1,5.6853,3187.500,6.3787,5062.50
8,015113cad,15,1,50.0533,93.750,53.3973,1125.00
9,0151b7d20,1,1,46.0320,3843.750,46.9280,5625.00


In [12]:
train_gby = train.groupby('recording_id').agg(
    {'species_id': list,
    'songtype_id': list,
    't_min': list,
    'f_min': list,
    't_max': list,
    'f_max': list})
train_gby

,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25]
006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4]
007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25]
0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04]
009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7]
...,...,...,...,...,...,...
fe8d9ac40,[13],[1],[53.472],[93.75],[54.096000000000004],[843.75]
fea6b438a,[4],[1],[43.5787],[2531.25],[45.7653],[4031.25]
ff2eb9ce5,[0],[1],[15.2267],[5906.25],[16.0213],[8250.0]


In [13]:
train_gby = train_gby.reset_index()
train_gby

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25]
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4]
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25]
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04]
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7]
...,...,...,...,...,...,...,...
1127,fe8d9ac40,[13],[1],[53.472],[93.75],[54.096000000000004],[843.75]
1128,fea6b438a,[4],[1],[43.5787],[2531.25],[45.7653],[4031.25]
1129,ff2eb9ce5,[0],[1],[15.2267],[5906.25],[16.0213],[8250.0]
1130,ffb8d8391,[5],[1],[14.3467],[4781.25],[16.6987],[10406.2]


In [14]:
train_gby['name'] = tat_path_list_df

In [15]:
train_gby.to_pickle(str(RAW_DATA / 'train_gby_mel4096.pkl'))
# tat_path_list_df.to_pickle(str(output_dir / 'train_gby_wavpath.pkl'))

In [16]:
train_gby

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25],/home/knikaido/work/Rainforest-Connection/Git/...
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4],/home/knikaido/work/Rainforest-Connection/Git/...
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25],/home/knikaido/work/Rainforest-Connection/Git/...
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04],/home/knikaido/work/Rainforest-Connection/Git/...
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7],/home/knikaido/work/Rainforest-Connection/Git/...
...,...,...,...,...,...,...,...,...
1127,fe8d9ac40,[13],[1],[53.472],[93.75],[54.096000000000004],[843.75],/home/knikaido/work/Rainforest-Connection/Git/...
1128,fea6b438a,[4],[1],[43.5787],[2531.25],[45.7653],[4031.25],/home/knikaido/work/Rainforest-Connection/Git/...
1129,ff2eb9ce5,[0],[1],[15.2267],[5906.25],[16.0213],[8250.0],/home/knikaido/work/Rainforest-Connection/Git/...
1130,ffb8d8391,[5],[1],[14.3467],[4781.25],[16.6987],[10406.2],/home/knikaido/work/Rainforest-Connection/Git/...
